# Van der Pol oscillator
We will look at the second order differentual equation (see https://en.wikipedia.org/wiki/Van_der_Pol_oscillator):

$$
{d^2y_0 \over dx^2}-\mu(1-y_0^2){dy_0 \over dx}+y_0= 0
$$

In [ ]:
from __future__ import division, print_function
import itertools
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from pyodesys import SymbolicSys
sp.init_printing()
%matplotlib inline
print(sp.__version__)

One way to reduce the order of our second order differential equation is to formulate a system of first order ODEs, using:

$$ y_1 = \dot y_0 $$

which gives us:

$$
\begin{cases}
\dot y_0 = y_1 \\
\dot y_1 = \mu(1-y_0^2) y_1-y_0
\end{cases}
$$

Let's call this system of ordinary differential equations vdp1:

In [ ]:
vdp1 = lambda x, y, p: [y[1], -y[0] + p[0]*y[1]*(1 - y[0]**2)]
mu_val = 2.5
y0_1 = [0, 1]
y0_1, (y0_1[0], vdp1(0, y0_1, [mu_val])[0])

An alternative would be to use use the Liénard transformation:

$$ y = x - x^3/3 - \dot x/\mu $$

In [ ]:
transf = lambda y, dydx, p: [y[0], y[0] - y[0]**3/3 - dydx[0]/p[0]]
x, mu = sp.symbols('x mu', real=True)
y = [yi(x) for yi in sp.symbols('y:2', cls=sp.Function)]
dydx = [yi.diff(x) for yi in y]
[sp.Eq(yi, expr, evaluate=False) for yi, expr in zip(y, transf(y, dydx, [mu]))]  # Just for displaying

which gives us (we could generate this result using SymPy):
$$
\begin{cases}
\dot y_0 = \mu \left(y_0-\frac{1}{3}y_0^3-y_1\right) \\
\dot y_1 = \frac{1}{\mu} y_0
\end{cases}
$$

In [ ]:
vdp2 = lambda x, y, p: [p[0]*(y[0] - y[0]**3/3 - y[1]), y[0]/p[0]]
y0_2 = transf(y0_1, vdp1(0, y0_1, [mu_val]), [mu_val])
(y0_2, y0_2[0], vdp2(0, y0_2, [mu_val])[0])

In [ ]:
def solve_and_plot(odesys, y0, tout, mu, solver='cvode', **kwargs):
    plt.figure(figsize=(16, 4))
    xout, yout, info = odesys.integrate(solver, tout, y0, [mu], **kwargs)
    plt.subplot(1, 2, 1)
    odesys.plot_result(ls=('-',), c=('k', 'r'))
    plt.legend()
    plt.subplot(1, 2, 2)
    odesys.plot_phase_plane()
    odesys.integrate('cvode', [0, tend], y0[::-1], [mu])
    odesys.plot_phase_plane(ls='--')
    return len(xout), info

In [ ]:
tend = 25

In [ ]:
odesys1 = SymbolicSys.from_callback(vdp1, 2, 1, names='y0 y1'.split())
odesys1.exprs

In [ ]:
solve_and_plot(odesys1, y0_1, np.linspace(0, tend, 500), mu_val, solver='cvode')

For large values of $\mu$ we run into trouble:

In [ ]:
for mu_ in [51, 52., 53.]:
    odesys1.integrate('cvode', [0, 25], [0, 1], [mu_])
    odesys1.plot_phase_plane(indices=(0,1), label=str(mu_))
plt.legend()

Same thing happens for our alternative formulation:

In [ ]:
odesys2 = SymbolicSys.from_callback(vdp2, 2, 1, names='y0 y1'.split())
odesys2.exprs

In [ ]:
solve_and_plot(odesys2, y0_2, tend, mu_val, solver='cvode')

In [ ]:
ls = itertools.cycle(('-', '--', ':'))
for mu_ in [10, 20, 30]:
    odesys2.integrate('cvode', [0, tend], y0_2, [mu_])
    odesys2.plot_phase_plane(indices=(0,1), label=str(mu_), ls=ls.next())
plt.legend()

## Audio
Plotting is instructive from a mathematical sense, but these equations were often investigated also from listening the the audio from electrical circuits:

In [ ]:
def arr_to_wav(arr, rate=44100):
    from IPython.display import Audio
    from scipy.io.wavfile import write
    scaled = np.int16(arr/np.max(np.abs(arr)) * 32767)
    write('test.wav', rate, scaled)
    return Audio('test.wav')

In [ ]:
xout, yout, info = odesys1.integrate('cvode', np.linspace(0, 3000, 2*44100), y0_1, [2.0])
arr_to_wav(yout[:, 0])

In [ ]:
def overlay(tend_mu, odesys=odesys1, time=3, rate=44100, plot=False):
    yout_tot = None
    for tend, mu in tend_mu:
        print(tend, mu)
        xout, yout, info = odesys.integrate('cvode', np.linspace(0, tend, time*rate), y0_1, mu)
        if yout_tot is None:
            yout_tot = yout[:, 0]
        else:
            yout_tot += yout[:, 0]
    if plot:
        plt.figure(figsize=(16,4))
        plt.plot(yout_tot)
    return arr_to_wav(yout_tot, rate=rate)

In [ ]:
overlay([
    (800, [2.0]),
    (900, [2.1]),
    (1600, [2.0]),
    (1700, [2.2]),
], plot=True)

## Forced van der pol oscillator

In [ ]:
vdp_forced = lambda x, y, p: [y[1], p[1]*sp.sin(p[2]*x) - y[0] + p[0]*y[1]*(1 - y[0]**2)]
odesys_forced = SymbolicSys.from_callback(vdp_forced, 2, 3)
overlay([(700, [8.53, 1.2, 0.62831853])], odesys_forced, plot=True)  # Chaotic behavior

### Transient $\mu$

In [ ]:
vdp_transient = lambda x, y, p: [y[1], - y[0] + p[0]*sp.exp(-p[1]*x)*y[1]*(1 - y[0]**2)]
odesys_transient = SymbolicSys.from_callback(vdp_transient, 2, 2)
odesys_transient.exprs

In [ ]:
overlay([
    (1700, [0.1, 1/3200.]),
    (1730, [0.5, 1/3200.]),
    (3430, [0.1, 2/6430.]),
    (3460, [0.5, 2/6430.]),
], odesys_transient, plot=True)